In [ ]:
!pip install kdbai_client --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 28.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.1.4 which is incompatible.
mizani 0.13.0 requires pandas>=2.2.0, but you have pandas 2.1.4 which is incompatible.
plotnine 0.14.1 requires pandas>=2.2.0, but you have pandas 2.1.4 which is incompatible.


In [ ]:
!git clone https://github.com/facebookresearch/ImageBind

Cloning into 'ImageBind'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 146 (delta 60), reused 39 (delta 39), pack-reused 58 (from 1)
Receiving objects: 100% (146/146), 2.64 MiB | 27.08 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [ ]:
import os
os.chdir('./ImageBind')

In [ ]:
!pip install . --quiet

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 57.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 49.7

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import os
import PIL
from PIL import Image
import torch
from imagebind import data
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)
print("")

  0%|          | 0.00/4.47G [00:00<?, ?B/s]

In [ ]:
import csv
def read_context(filename):
  context = {}
  with open(filename, 'r') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)
    for row in csv_reader:
      context[row[0]] = row[1]
  return context

context = read_context('/content/drive/MyDrive/Persons/context.csv')

In [ ]:
def getEmbeddingVector(inputs):
    with torch.no_grad():
        embedding = model(inputs)
    for key, value in embedding.items():
        vec = value.reshape(-1)
        vec = vec.cpu().numpy()
        return(vec)

def dataToEmbedding(dataIn,dtype):
    if dtype == 'image':
        data_path = [dataIn]
        inputs = {
          ModalityType.VISION: data.load_and_transform_vision_data(data_path, device)
        }
    elif dtype == 'text':
        txt = [dataIn]
        inputs = {
          ModalityType.TEXT: data.load_and_transform_text(txt, device)
        }
    vec = getEmbeddingVector(inputs)
    return(vec)

def queryToEmbedding(dataIn):
    data_path = [dataIn]
    inputs = {
        ModalityType.VISION: data.load_and_transform_vision_data(data_path, device)
    }
    vec = getEmbeddingVector(inputs)
    return(vec)

In [ ]:
columns = ['path', 'embeddings', 'context']
df = pd.DataFrame(columns=columns)

In [ ]:
!pip install pyheif --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 40.1 MB/s eta 0:00:00


In [ ]:
import os
import pyheif
from PIL import Image
from tqdm.auto import tqdm
def convert_heic_to_jpg(input_path, output_path):
    heif_file = pyheif.read(input_path)
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
        heif_file.mode,
        heif_file.stride,
    )
    image.save(output_path, format="JPEG")

input_dir = '/content/drive/MyDrive/LLM_Mid1/images/'
output_dir = '/content/drive/MyDrive/RAG/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for image_name in tqdm(os.listdir(input_dir)):
    image_path = os.path.join(input_dir, image_name)
    try:
        with open(image_path, 'rb') as f:
            header = f.read(12)
            if b'ftypheic' in header:
                output_path = os.path.join(output_dir, image_name.replace('.jpeg', '_converted.jpg'))
                convert_heic_to_jpg(image_path, output_path)
    except Exception as e:
        print(f"Error handling {image_name}: {e}")

  0%|          | 0/282 [00:00<?, ?it/s]

In [ ]:
from tqdm.auto import tqdm
for image in tqdm(os.listdir('/content/drive/MyDrive/Persons/Images')):
    path = f'/content/drive/MyDrive/Persons/Images/{image}'
    name = image.split('.')[0]
    print(name)
    con = context[name]
    if con == '':
        continue
    embedding = dataToEmbedding(path, 'image')
    new_row = {'path': path,
               'embeddings':embedding,
               'context':con}
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
import os
from getpass import getpass
import kdbai_client as kdbai
import time

In [ ]:
KDBAI_ENDPOINT = '<KDBAI ENDPOINT>'
KDBAI_API_KEY = 'KDBAI API KEY'

In [ ]:
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

In [ ]:
db = session.database('default')

In [ ]:
# try:
#     db.table("LLM").drop()
#     time.sleep(5)
# except kdbai.KDBAIException:
#     pass

In [ ]:
# schema = [
# {'name': 'path', 'type': 'str'},
# {'name': 'context', 'type': 'str'},
# {'name': 'embeddings', 'type': 'float64s'}]

# indexes = [ {'name': 'vectorIndex', 'type': 'flat',
# 'params': {'dims': 1024, 'metric': 'CS'},
# 'column': 'embeddings'}]
table = db.table('LLM')

In [ ]:
from tqdm import tqdm
n = 2000
for i in tqdm(range(0, df.shape[0], n)):
    table.insert(df[i:i+n].reset_index(drop=True))

100%|██████████| 1/1 [00:00<00:00,  2.04it/s]
